In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import labolibrary as labo
import pickle
from sklearn.preprocessing import RobustScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [2]:

#DATOS_DIR = '~/buckets/b1/datasets/'
DATOS_DIR = '../data/'      

scalers = {}
# Function to center, scale, and return a series
def scale_group(group):
    scaler = RobustScaler()
    scaled_values = scaler.fit_transform(group.values.reshape(-1, 1)).flatten()
    scalers[group.name] = scaler  # Store the scaler for this group
    return pd.Series(scaled_values, index=group.index, name=group.name)

# Function to inverse transform (de-scale) and decenter, and return a series
def inverse_scale_group(group):
    group_name = group.name
    scaler = scalers[group_name]
    inversed_centered_values = scaler.inverse_transform(group.values.reshape(-1, 1)).flatten()
    original_values = inversed_centered_values
    return pd.Series(original_values, index=group.index, name=group_name)


In [3]:

# Leer datos
#df_final = pd.read_parquet(DATOS_DIR+'FE_dataset-CARLA.parquet') 
df_final = pd.read_parquet(DATOS_DIR+'/FE_dataset-PCA-Decompose.parquet') 
df_final.columns = df_final.columns.str.replace(' ', '_').str.replace(r'[^A-Za-z0-9_]', '', regex=True)




In [4]:
#weight = df_final.groupby(['product_id', df_final.index])['weight'].transform('mean')

df_final.groupby('product_id')['weight'].transform(scale_group)


periodo
2018-01     2.908873
2018-01     0.091007
2018-01     0.363669
2018-01     0.181895
2018-01     5.999520
             ...    
2019-11    -0.307798
2019-11    20.771567
2019-11     1.538647
2019-11    -0.076950
2019-11    -0.230849
Freq: M, Name: weight, Length: 1493991, dtype: float64

In [5]:

### Filtrar datos
df_true = df_final.loc['2019-12-01':'2020-01-01']
df_final = df_final.loc['2018-01-01':'2019-11-01']


#Filtro de no compradores
#Step 1: Ensure the index is a datetime type
df_final.index = df_final.index.to_timestamp()
# Step 2: Determine the last date and calculate the date 3 months prior
ls_date  = df_final.index.max()
three_months_prior = ls_date - pd.DateOffset(months=3)

# Step 3: Filter the dataframe to include onl+y rows within the last 3 months
last_3_months_df = df_final[df_final.index >= three_months_prior]

# Step 4: Identify the unique client_id that have purchased within this period
active_clients = last_3_months_df['customer_id'].unique()

# Step 5: Filter the original dataframe to include only these client_id
df_final = df_final[df_final['customer_id'].isin(active_clients)]

df_final.index = pd.PeriodIndex(df_final.index, freq='M')


#Filtro test
#df_final = df_final[df_final['product_id'] < 20013]

weight= df_final[['weight','product_id']]
df_final.drop(columns=['weight'], inplace=True)


In [6]:
# Correr Modelo
params={
        'boosting_type': 'gbdt',
        'objective': 'Regression',
        'metric':'rmse',
        'verbose': -1,
        #'n_jobs': -1,
        #'seed': 113,
        #'learning_rate': 0.2,
        #'bagging_fraction': 0.85,
        #'bagging_freq': 1, 
        #'colsample_bytree': 0.85,
        #'colsample_bynode': 0.85,
        #'min_data_per_leaf': 25,
        #'num_leaves': 200,
        #'lambda_l1': 0.5,
        #'lambda_l2': 0.5
}

predictions_all = pd.DataFrame(columns=['tn'])
products = df_final['product_id'].unique()
tot = len(products)
nro = 0
for producto in products:
    print(f'Fitting and predicting for product_id: {producto}')
    # Filtrar los datos del producto
    df_producto = df_final[df_final['product_id'] == producto]
    weight_p= weight[weight['product_id'] == producto]
    # Prepare data for LSTM on tn_2 only
    X = df_producto[['tn_2']].values.astype('float32')
    y = df_producto['tn_2'].values.astype('float32')
    X = X.reshape((X.shape[0], 1, X.shape[1]))
    #######################################################    
    # Define LSTM model
    model = Sequential()
    model.add(LSTM(50, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy')

    # Train LSTM model
    model.fit(X, y, epochs=10, batch_size=32, verbose=1)

    # Extract features from LSTM
    features = model.predict(X)

    # Prepare data for LightGBM
    # Convert LSTM features to DataFrame
    features_df = pd.DataFrame(features, index=df_producto.index, columns=[f'lstm_feature_{i}' for i in range(features.shape[1])])
    df_producto = pd.concat([df_producto, features_df], axis=1)
    df_producto['tn_2'] = y
    
    #############################################################################  
    #### Agrupar y escalar
   
    model, average_metric = labo.train_lightgbm_model(df_producto,params,metric='rmse',
                                                      weights=weight_p['weight'])
    print("Overall rmse metric: ", average_metric)
    # Predict values for the entire dataset using the trained models
    # Prepare last data points for prediction
    last_data_points = df_producto[df_producto.index == df_producto.index.max()].copy()
    last_data_points.drop(columns=['tn_2'], inplace=True)
    # Predict the next month's value using the trained model
    predictions = labo.predict_next_month(model, last_data_points)
    preds = predictions.groupby('product_id')['tn_2'].transform(inverse_scale_group)
    predictions['tn'] = preds
    predictions.drop(columns=['tn_2'], inplace=True)
    predictions = predictions.reset_index()
    predictions =  predictions.groupby('product_id')['tn'].sum()
    predictions.columns = ['product_id', 'tn']
    predictions_all = pd.concat([predictions_all, predictions])
    print(predictions_all[-1:])


Fitting and predicting for product_id: 20524
Epoch 1/10


d:\Dropbox\Python\LaboIII\labo3-2024v\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 857us/step - loss: 0.7601
Epoch 2/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 690us/step - loss: -0.5529
Epoch 3/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 757us/step - loss: -2.5160
Epoch 4/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step - loss: -4.3655
Epoch 5/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 791us/step - loss: -7.0271
Epoch 6/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 786us/step - loss: -8.9593
Epoch 7/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 791us/step - loss: -13.8028
Epoch 8/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 778us/step - loss: -14.9121
Epoch 9/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 721us/step - loss: -18.6335
Epoch 10/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 771us/step - loss: -26.3106
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[22]	validation's rmse: 2.31617
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[23]	validation's rmse: 3.12
Training until validation scores do

C:\Users\Admin\AppData\Local\Temp\ipykernel_920\2959756526.py:70: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predictions_all = pd.concat([predictions_all, predictions])
C:\Users\Admin\AppData\Local\Temp\ipykernel_920\2959756526.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictions_all = pd.concat([predictions_all, predictions])
d:\Dropbox\Python\LaboIII\labo3-2024v\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Seque

68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 649us/step - loss: 0.0926  
Epoch 2/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -2.7596
Epoch 3/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 864us/step - loss: -6.1397
Epoch 4/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 789us/step - loss: -9.7359
Epoch 5/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 845us/step - loss: -13.7876
Epoch 6/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 719us/step - loss: -21.0950
Epoch 7/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 710us/step - loss: -23.3329
Epoch 8/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 687us/step - loss: -32.2999
Epoch 9/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 655us/step - loss: -37.1725
Epoch 10/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 762us/step - loss: -40.5668
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 594us/step
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[59]	validation's rmse: 3.71472
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[32]	validation's rmse: 3.13694
Training until validation sc

...

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 970us/step - loss: 1.0024
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 877us/step - loss: 0.6806
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 854us/step - loss: 0.5179
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3557 
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 992us/step - loss: 0.0897
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 982us/step - loss: -0.2542
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 936us/step - loss: -0.6251
Epoch 8/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 866us/step - loss: -0.4395
Epoch 9/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 792us/step - loss: -0.6416
Epoch 10/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 956us/step - loss: -1.9719
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[47]	validation's rmse: 1.44178
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[113]	validation's rmse: 3.91404
Training until validation scores don't improve for 10 rounds


d:\Dropbox\Python\LaboIII\labo3-2024v\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.7053  
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5914 
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 962us/step - loss: 0.4617
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 964us/step - loss: 0.4078
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 958us/step - loss: 0.3459
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2002 
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1633 
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -0.1889 
Epoch 9/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -0.0578
Epoch 10/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -0.4725 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	validation's rmse: 1.33755
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[33]	validation's rmse: 4.96293
Training until validation scores don't improve for 10 rounds
Early stop

d:\Dropbox\Python\LaboIII\labo3-2024v\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.6866  
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 939us/step - loss: 0.3805
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0934 
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 889us/step - loss: -0.2690
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 950us/step - loss: -0.5222
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 891us/step - loss: -0.7544
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -0.5260
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 867us/step - loss: -1.3440
Epoch 9/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 870us/step - loss: -1.9147
Epoch 10/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 950us/step - loss: -1.4591
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	validation's rmse: 5.87274
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	validation's rmse: 2.93301
Training until validation scores don't improve for 10 rounds
Ear

In [8]:

predictions_all['tn']=predictions_all['tn'].astype('float32')
predictions_all.index.names = ['product_id']
predictions_all.to_csv(DATOS_DIR+'/pred/0020-prediccion-rmse_scaled_CORRECT2-product_id_LSTM-No_Buyer_weights_PCA_Decompose.csv', index=True,header=True)
print("Overall custom metric: ", average_metric)


Overall custom metric:  1.1392443077593304
